Benchmarking

In [1]:
from itertools import product

import numpy as np
from qibo import gates
from qibo.backends import NumpyBackend
from qibo.quantum_info import fidelity, trace_distance

from qibo.quantum_info import random_density_matrix
from sympy import Matrix


SINGLE_QUBIT_BASIS = ["X", "Y", "Z"]

""" 
The order of the basjs for the reconstruction of the density matrix is set with TWO_QUBIT BASIS.
This will be used in:
_acquisition for the calculation of the rotated density matrix rho' 
that will lead to the probability vector 'probabilities'.
_fit for the construction of the matrix measurement.
"""
TWO_QUBIT_BASIS = list(product(SINGLE_QUBIT_BASIS, SINGLE_QUBIT_BASIS)) 

In [2]:
def rotation_matrix(basis):
    """Matrix of the gate implementing the rotation to the given basis.

    Args:
        basis (str): One of Pauli basis: X, Y or Z.
    """
    backend = NumpyBackend()
    if basis == "Z":
        return np.eye(2, dtype=complex)
    return getattr(gates, basis)(0).basis_rotation().matrix(backend) 
"""
getattr(gates, basis)(0) this is a qibo gate object that represents a Pauli matrix.
method basis_rotation() returns the rotation matrix associated with the gate (still not a matrix)
matrix(backend) returns the matrix representation of the gate.
"""

# Calculate the probability vector rho'= R rho R^dagger
# where R is the rotation matrix to the measurement basis 
# and rho is the density matrix we are providing
def _acquisition(density_matrix):
    # Calculates the list of rotation matrices:
    rotations = [
        np.kron(rotation_matrix(basis1), rotation_matrix(basis2))
        for basis1, basis2 in TWO_QUBIT_BASIS
    ]
    probabilities_list = []
    for rotation in rotations:
        # Calculate the rotated density matrix rho' = R rho R^dagger
        density_matrix_rotated = rotation @ density_matrix @ np.conj(rotation).T
        # Calculate the probabilities (diagonal of the rotated density matrix rho')
        probabilities = np.real(np.diag(density_matrix_rotated)) # probability vector, must be real
        probabilities_list.append(probabilities)

    return probabilities_list


def _fit(probabilities_list):
    """Post-processing for two qubit state tomography.

    Uses a linear inversion algorithm to reconstruct the density matrix
    from probabilities, with the following steps:
    1. Construct a linear transformation M, from density matrix
    to Born-probabilities in the space of all two-qubit measurement bases
    (in our case XX, XY, XZ, YX, YY, YZ, ZX, ZY, ZZ).
    2. Invert M to get the transformation from Born-probabilities to
    density matrices.
    3. Map this vector to a density matrix (``measured_raw_density_matrix``) using the
    inverse of M from step 2.
    """
    rotations = [
        np.kron(rotation_matrix(basis1), rotation_matrix(basis2))
        for basis1, basis2 in TWO_QUBIT_BASIS
    ]

    # Construct the linear transformation from density matrix to Born-probabilities
    measurement = np.zeros((0, 16))
    for rotation in rotations:
        channel = np.kron(rotation, np.conj(rotation))
        measure_channel = channel[np.eye(4, dtype=bool).flatten(), :]
        measurement = np.concatenate((measurement, measure_channel))

    # Invert to get linear transformation from Born-probabilities to density matrix
    inverse_measurement = np.linalg.pinv(measurement)

    # Reshape the probabilities list (9 vectors of 4 elementos) to one vector:
    """
    Analogous to the following code, we can reshape probabilities_list from a list
    of 9 vectors with 4 elements to a single vector with 36 elements:
    probabilities_list = np.array(probabilities_list)
    probabilities = probabilities_list.reshape((36,1))
    """
    probabilities = np.zeros((36,1))
    for ib in range(9):
        for i in range(4):
            # ib = 0,1,2,...,8 associated to each basis.
            # for each basis: 0->XX, 1->XY, 2->XZ, ... 
            # The order is already determined from the construction of probability list (_acquisition)
            # and it is the same order that is being used in the reconstruction of measurement.
            probabilities[ib*4+i] = probabilities_list[ib][i]

    # Reconstruction:
    measured_vectorized_density_matrix = inverse_measurement.dot(probabilities)
    measured_density_matrix = measured_vectorized_density_matrix.reshape((4, 4))

    return measured_density_matrix


$\textbf{Testing the code for an specific case}$

In [3]:
# Input of _acquisition:
density_matrix = random_density_matrix(4)
# Output of _acquisition and input of _fit:
probabilities_list = _acquisition(density_matrix)
# Output of _fit:
measured_density_matrix = _fit(probabilities_list)


fidelity_value = fidelity(density_matrix, measured_density_matrix)
trace_distance_value = trace_distance(density_matrix, measured_density_matrix)

[Qibo 0.2.16|INFO|2025-04-24 12:41:37]: Using numpy backend on /CPU:0


Comparing the density matrix used to feed the code and the resulting density matrix, we can see they are the same:

In [4]:
fidelity_value

0.9999999999999996

In [5]:
trace_distance_value

5.323449685047958e-16

$\textbf{Testing the code for 10000 random initial states}$

In [6]:
# Generate n random density matrix for a 2-qubit system
density_matrices = [random_density_matrix(4) for _ in range(10000)]

# Run the linear inversion tomography multiple times
fidelity_list = []
trace_distance_list = []
measured_density_matrix_list = [] # Store the measured density matrices
for density_matrix in density_matrices:
    probabilities_list = _acquisition(density_matrix)
    measured_density_matrix = _fit(probabilities_list)
    # Store the measured density matrix
    measured_density_matrix_list.append(measured_density_matrix)
    # Calculate fidelity and trace distance
    fidelity_value = fidelity(density_matrix, measured_density_matrix)
    trace_distance_value = trace_distance(density_matrix, measured_density_matrix)
    fidelity_list.append(fidelity_value)
    trace_distance_list.append(trace_distance_value)

As before, we can check that the fidelities are 1 and the trace distances are 0:

In [7]:
fidelity_list

[0.9999999999999993,
 1.0000000000000002,
 1.000000000000001,
 1.0000000000000016,
 0.9999999999999991,
 0.9999999999999991,
 0.9999999999999971,
 0.9999999999999967,
 1.000000000000002,
 1.0000000000000098,
 0.9999999999999916,
 1.000000000000001,
 0.999999999999999,
 0.9999999999999949,
 1.0,
 0.9999999999999982,
 0.9999999999999999,
 0.9999999999999972,
 0.9999999999999892,
 1.0000000000000002,
 0.9999999999999992,
 1.000000000000001,
 1.0000000000000004,
 0.9999999999999984,
 1.0000000000000007,
 0.9999999999999982,
 1.0000000000000007,
 1.0000000000000007,
 1.0,
 0.9999999999999984,
 0.9999999999999991,
 1.0000000000000004,
 0.9999999999999908,
 0.9999999999999992,
 1.0000000000000013,
 1.0000000000000009,
 0.999999999999996,
 1.0000000000000067,
 0.9999999999999865,
 0.999999999999985,
 1.0000000000000004,
 0.9999999999999956,
 0.9999999999999833,
 0.9999999999999993,
 1.000000000000001,
 1.0000000000000007,
 0.9999999999999993,
 0.9999999999999971,
 0.9999999999999991,
 0.999999

In [8]:
trace_distance_list

[8.480838344211645e-16,
 4.38498694023118e-16,
 8.893331132204198e-16,
 4.869223500155252e-16,
 1.0396031642433376e-15,
 5.566947173472695e-16,
 5.532650667147923e-16,
 8.229774687370176e-16,
 6.288741248564992e-16,
 6.298299844601015e-16,
 6.732905513973394e-16,
 9.71971300934924e-16,
 5.351085046608871e-16,
 9.138229060115524e-16,
 5.909978333180373e-16,
 5.973227227484137e-16,
 6.023441830388785e-16,
 8.293820107864118e-16,
 8.751125713036479e-16,
 6.435376923462878e-16,
 6.181323357130772e-16,
 6.941173849193406e-16,
 6.633917328703081e-16,
 6.851175858790101e-16,
 4.993831122296401e-16,
 6.590796393449717e-16,
 6.375562349597877e-16,
 4.3259633431350725e-16,
 4.276239384026656e-16,
 5.004655138269751e-16,
 7.937494969539141e-16,
 1.0827065703452535e-15,
 8.630152810284059e-16,
 6.853990680996165e-16,
 1.1302233673912745e-15,
 6.288845972488221e-16,
 6.217051075670278e-16,
 7.708457359556851e-16,
 1.0776699322504545e-15,
 8.345284897952739e-16,
 6.42444525608706e-16,
 6.66055157788

Threshold for fidelity and trace distance:

In [9]:
"""
Note: The threshold value depends on the number of shots (nshots).
If the number of shots is changed, this value must be adjusted accordingly.
Ensure the threshold is low enough to consider the result acceptable.
"""
def check_fidelity(fidelity_list):
    for fidelity in fidelity_list:
        if abs(fidelity - 1) > 1e-10:
            return False
    return True

In [10]:
check_fidelity(fidelity_list)

False

We find monopoles when running the test.
To print the density matrix and fidelities associated with the monopoles:

In [11]:
# Find the indices where abs(fidelity - 1) > 1e-10
indices = [i for i, fidelity in enumerate(fidelity_list) if abs(fidelity - 1) > 1e-10] # Estaba en 1e-10

# Extract the density matrices associated with those indices
monopole_density_matrices = [measured_density_matrix_list[i] for i in indices]

# Extract the fidelities associated with the monopoles
monopole_fidelities = [fidelity_list[i] for i in indices]

# Print the results:
print("Indices with unacceptable fidelity:", indices)

print("Fidelities of the monopoles:")
for i, fidelity in zip(indices, monopole_fidelities):
    print(f"Index {i}: {fidelity}")

print("Associated density matrices:")
for i, matrix in zip(indices, monopole_density_matrices):
    print(f"Index {i}:")
    print(matrix)


Indices with unacceptable fidelity: [151, 380, 708, 987, 1032, 1115, 1172, 1276, 1340, 1471, 1734, 1951, 1992, 2074, 2101, 2187, 2413, 2962, 3324, 3383, 3459, 3531, 3734, 3902, 3955, 3976, 4089, 4149, 4227, 4393, 4871, 4973, 5221, 5312, 5383, 5398, 5423, 5593, 5742, 6276, 6299, 6686, 6741, 6867, 7058, 7068, 7121, 7675, 7804, 7905, 8006, 8033, 8087, 8291, 8322, 8374, 8624, 8864, 8941, 9374, 9572, 9726, 9731, 9743]
Fidelities of the monopoles:
Index 151: 0.999940367417488
Index 380: 0.9999808618632893
Index 708: 0.999997540009084
Index 987: 0.9999483074009152
Index 1032: 0.9999203066436734
Index 1115: 0.9999953766723527
Index 1172: 0.9999724996346671
Index 1276: 0.9999794552698715
Index 1340: 0.9999771385460302
Index 1471: 0.9999722622456382
Index 1734: 0.9999233179493765
Index 1951: 0.9999943833811685
Index 1992: 0.9999913540479134
Index 2074: 0.9999499691220816
Index 2101: 0.9999984084464736
Index 2187: 0.9999343582941969
Index 2413: 0.9999689254386206
Index 2962: 0.9999793123978977
In

In [12]:
# Goes through the monopole density matrices and print the ones with fidelity < 0.9999:
found = False  

for i, fidelity_value in enumerate(monopole_fidelities):
    if fidelity_value < 0.9999001:
        found = True  
        print(f"Index {i}: Fidelity = {fidelity_value}")
        print("Associated density matrix:")
        print(monopole_density_matrices[i])

# If no density matrix with fidelity < 0.999 is found, print a message:
if not found:
    print("There is no density matrix with fidelity < 0.9999.")


There is no density matrix with fidelity < 0.9999.


.

In [13]:
from qibo.quantum_info import fidelity # Necessary to reimport since a variable called fidelity has overwritten the function.
fidelity(monopole_density_matrices[0], density_matrices[33])

0.7492608573141021

In [14]:
# Trace density matrix:
#np.trace(density_matrices[33]@density_matrices[33])
from sympy import Matrix

# Convertir la matriz de NumPy a una matriz de SymPy y luego imprimirla
print(Matrix(density_matrices[33]))


Matrix([[0.398358422228258, 0.0103323652490661 + 0.0607806288301734*I, -0.175711553196048 - 0.114113095756986*I, 0.0695290277243567 - 0.0116773868062768*I], [0.0103323652490661 - 0.0607806288301734*I, 0.0649088395740500, -0.043921909823709 - 0.0196368608896399*I, 0.0298207012160262 + 0.0268241668424571*I], [-0.175711553196048 + 0.114113095756986*I, -0.043921909823709 + 0.0196368608896399*I, 0.291505810929925, -0.115267664372295 - 0.0439411476109013*I], [0.0695290277243567 + 0.0116773868062768*I, 0.0298207012160262 - 0.0268241668424571*I, -0.115267664372295 + 0.0439411476109013*I, 0.245226927267767]])


In [15]:
print(Matrix(monopole_density_matrices[0]))

Matrix([[0.226738506097635 + 5.07159127833262e-20*I, -0.0195605147856061 + 0.11966222320026*I, 0.0552085605405483 + 0.0277794855511137*I, 0.03562847105201 - 0.0246374699400774*I], [-0.0195605147856058 - 0.11966222320026*I, 0.330886570358993 - 2.09942793650583e-17*I, 0.0726829774011671 + 0.0789865258159017*I, 0.0336003607808289 - 0.251876653372035*I], [0.0552085605405483 - 0.0277794855511135*I, 0.0726829774011667 - 0.0789865258159015*I, 0.1632927153412 + 4.18678499542719e-17*I, -0.0768659502382994 - 0.0189447778320128*I], [0.0356284710520097 + 0.0246374699400779*I, 0.0336003607808288 + 0.251876653372035*I, -0.0768659502382994 + 0.0189447778320125*I, 0.279082208202172 + 7.94724777983326e-18*I]])


In [16]:
#np.trace(monopole_density_matrices[0] @ monopole_density_matrices[0])

In [17]:
from qibo.quantum_info import fidelity
fidelity(density_matrices[64], monopole_density_matrices[0])

0.7532729493921473

In [18]:
# Calculation of the fidelity to check if we get the same as qibo or not.
# qibo fidelity results:
# fidelity(density_matrices[33], monopole_density_matrices[0])= 0.9999017711797435

import numpy as np
from scipy.linalg import sqrtm


def fidelity_test(sigma, rho):
    # Compute the square root of rho
    sigma_sqrt = sqrtm(sigma)
    
    # Compute the product of the square root of sigma and rho
    product = sigma_sqrt @ rho @ sigma_sqrt

    # Ensure the product is Hermitian
    product = 0.5 * (product + product.conj().T)

    # Compute the eigenvalues of the product
    eigenvalues = np.linalg.eigh(product)[0]
    
    # Sum the square roots of the eigenvalues
    fid = np.sum(np.sqrt(np.maximum(eigenvalues, 0.0)))
    
    return fid

fidelity_test(density_matrices[33], monopole_density_matrices[0])


0.7492608573140995

In [19]:
abs(fidelity(density_matrices[33], monopole_density_matrices[0]) - fidelity_test(density_matrices[33], monopole_density_matrices[0]))

3.552713678800501e-15

In [20]:
# Check if the fidelity_test is correct with a specific example:
A = np.array([[1, 0], [0, 0]])
B = np.array([[0.5, 0], [0, 0.5]])
fidelity_test(A,B)

0.7071067811865476

The value 0.7071067811865476 corresponds to the one calculated analitically.

In [21]:
from qibo.quantum_info import fidelity
A = np.array([[1, 0], [0, 0]])
B = np.array([[0.5, 0], [0, 0.5]])
fidelity(A, B)

0.5

It seems that qibo is taking the square in this case, since 0.7071067811865476 * 0.7071067811865476 = 0.5. But if we squared 0.9999999999998024, it does not give us qibo result 0.9999017711797435. There is an error in the fidelity code.

CHECKS:

In [22]:
from sympy import Matrix

# Convertir la matriz de NumPy a una matriz de SymPy y luego imprimirla
print(Matrix(density_matrices[200]))

Matrix([[0.110734458797220, 0.0646105089331112 - 0.0991518471607451*I, -0.0453863265349453 + 0.0603882142676403*I, -0.0655317321329057 - 0.110118780305475*I], [0.0646105089331112 + 0.0991518471607451*I, 0.363927314437240, 0.0784581050926286 - 0.0405955990765691*I, 0.0584971708467693 - 0.138568731521392*I], [-0.0453863265349453 - 0.0603882142676403*I, 0.0784581050926286 + 0.0405955990765691*I, 0.211354513601791, 0.00543342891322428 + 0.0421533624237042*I], [-0.0655317321329057 + 0.110118780305475*I, 0.0584971708467693 + 0.138568731521392*I, 0.00543342891322428 - 0.0421533624237042*I, 0.313983713163750]])


In [23]:
from sympy import Matrix

# Convertir la matriz de NumPy a una matriz de SymPy y luego imprimirla
print(Matrix(monopole_density_matrices[0]))

Matrix([[0.226738506097635 + 5.07159127833262e-20*I, -0.0195605147856061 + 0.11966222320026*I, 0.0552085605405483 + 0.0277794855511137*I, 0.03562847105201 - 0.0246374699400774*I], [-0.0195605147856058 - 0.11966222320026*I, 0.330886570358993 - 2.09942793650583e-17*I, 0.0726829774011671 + 0.0789865258159017*I, 0.0336003607808289 - 0.251876653372035*I], [0.0552085605405483 - 0.0277794855511135*I, 0.0726829774011667 - 0.0789865258159015*I, 0.1632927153412 + 4.18678499542719e-17*I, -0.0768659502382994 - 0.0189447778320128*I], [0.0356284710520097 + 0.0246374699400779*I, 0.0336003607808288 + 0.251876653372035*I, -0.0768659502382994 + 0.0189447778320125*I, 0.279082208202172 + 7.94724777983326e-18*I]])


In [24]:
from sympy import Matrix

# Convertir la matriz de NumPy a una matriz de SymPy y luego imprimirla
print(Matrix(density_matrices[200] - monopole_density_matrices[0]))

Matrix([[-0.116004047300415 - 5.07159127833262e-20*I, 0.0841710237187173 - 0.218814070361005*I, -0.100594887075494 + 0.0326087287165266*I, -0.101160203184916 - 0.0854813103653974*I], [0.084171023718717 + 0.218814070361005*I, 0.0330407440782465 + 2.09942793650583e-17*I, 0.00577512769146146 - 0.119582124892471*I, 0.0248968100659404 + 0.113307921850643*I], [-0.100594887075494 - 0.0326087287165268*I, 0.00577512769146188 + 0.119582124892471*I, 0.0480617982605911 - 4.18678499542719e-17*I, 0.0822993791515237 + 0.061098140255717*I], [-0.101160203184915 + 0.0854813103653969*I, 0.0248968100659405 - 0.113307921850643*I, 0.0822993791515237 - 0.0610981402557166*I, 0.0349015049615775 - 7.94724777983326e-18*I]])


In [25]:
print(monopole_density_matrices[0])

[[ 0.22673851+5.07159128e-20j -0.01956051+1.19662223e-01j
   0.05520856+2.77794856e-02j  0.03562847-2.46374699e-02j]
 [-0.01956051-1.19662223e-01j  0.33088657-2.09942794e-17j
   0.07268298+7.89865258e-02j  0.03360036-2.51876653e-01j]
 [ 0.05520856-2.77794856e-02j  0.07268298-7.89865258e-02j
   0.16329272+4.18678500e-17j -0.07686595-1.89447778e-02j]
 [ 0.03562847+2.46374699e-02j  0.03360036+2.51876653e-01j
  -0.07686595+1.89447778e-02j  0.27908221+7.94724778e-18j]]


In [26]:
print(density_matrices[200])

[[ 0.11073446+0.j          0.06461051-0.09915185j -0.04538633+0.06038821j
  -0.06553173-0.11011878j]
 [ 0.06461051+0.09915185j  0.36392731+0.j          0.07845811-0.0405956j
   0.05849717-0.13856873j]
 [-0.04538633-0.06038821j  0.07845811+0.0405956j   0.21135451+0.j
   0.00543343+0.04215336j]
 [-0.06553173+0.11011878j  0.05849717+0.13856873j  0.00543343-0.04215336j
   0.31398371+0.j        ]]


Check the eigenvalues of both the ideal and monopole:

In [27]:
np.linalg.eigvals(density_matrices[200])

array([0.56433887+4.28954965e-17j, 0.0068727 +6.18053463e-18j,
       0.29877023+5.72178073e-18j, 0.13001819-1.31644484e-17j])

In [28]:
np.linalg.eigvals(monopole_density_matrices[0])

array([6.23472661e-01+2.06050767e-16j, 2.63667955e-01-2.13403495e-18j,
       5.96325825e-05-9.32075131e-17j, 1.12799751e-01-8.29029270e-17j])

In [29]:
np.linalg.eigvals(density_matrices[200] - monopole_density_matrices[0]) # check

array([-0.36923402-1.32562514e-16j,  0.32317076-1.23688490e-17j,
       -0.07750065-1.32106244e-16j,  0.12356391+2.47496591e-16j])

In [30]:
Matrix(density_matrices[200] - monopole_density_matrices[0])

Matrix([
[-0.116004047300415 - 5.07159127833262e-20*I,    0.0841710237187173 - 0.218814070361005*I,   -0.100594887075494 + 0.0326087287165266*I,   -0.101160203184916 - 0.0854813103653974*I],
[    0.084171023718717 + 0.218814070361005*I, 0.0330407440782465 + 2.09942793650583e-17*I,   0.00577512769146146 - 0.119582124892471*I,    0.0248968100659404 + 0.113307921850643*I],
[  -0.100594887075494 - 0.0326087287165268*I,   0.00577512769146188 + 0.119582124892471*I, 0.0480617982605911 - 4.18678499542719e-17*I,    0.0822993791515237 + 0.061098140255717*I],
[  -0.101160203184915 + 0.0854813103653969*I,    0.0248968100659405 - 0.113307921850643*I,   0.0822993791515237 - 0.0610981402557166*I, 0.0349015049615775 - 7.94724777983326e-18*I]])

In [31]:
# Trace distance:
trace_distance(density_matrices[200], monopole_density_matrices[0])#check

0.44673466563347364

In [32]:
#fidelity(density_matrices[163], monopole_density_matrices[0]) # check
#type(density_matrices[163])
type(monopole_density_matrices[0])

numpy.ndarray

In [33]:
Monopole=[[0.256110022268168 + 1.94171044449499e-17*1j, 0.0778223813830341 - 0.237679296038382*1j, 0.00354139423033569 + 0.0455484061963264*1j, 0.0239438244520001 + 0.0399099865603644*1j], [0.0778223813830339 + 0.237679296038382*1j, 0.279060292026263 - 2.53439111481706e-17*1j, 0.0448513085614787 + 0.0308424368651327*1j, -0.0389519522138113 + 0.0173706952499513*1j], [0.00354139423033535 - 0.0455484061963262*1j, 0.044851308561479 - 0.0308424368651324*1j, 0.309139999138457 - 5.06226544085774e-17*1j, 0.0806763612869424 - 0.0157206204456681*1j], [0.023943824452 - 0.0399099865603646*1j, -0.038951952213811 - 0.0173706952499514*1j, 0.0806763612869426 + 0.0157206204456681*1j, 0.155689686567112 + 6.3066679539143e-17*1j]]
Ideal=[[0.256110022268168, 0.0778223813830341 - 0.237679296038382*1j, 0.00354139423033552 + 0.0455484061963264*1j, 0.0239438244520003 + 0.0399099865603645*1j], [0.0778223813830341 + 0.237679296038382*1j, 0.279060292026263, 0.044851308561479 + 0.0308424368651326*1j, -0.0389519522138111 + 0.0173706952499512*1j], [0.00354139423033552 - 0.0455484061963264*1j, 0.044851308561479 - 0.0308424368651326*1j, 0.309139999138457, 0.0806763612869426 - 0.0157206204456682*1j], [0.0239438244520003 - 0.0399099865603645*1j, -0.0389519522138111 - 0.0173706952499512*1j, 0.0806763612869426 + 0.0157206204456682*1j, 0.155689686567112]]
fidelity(Ideal, Monopole) # check

AttributeError: 'list' object has no attribute 'dtype'

In [ ]:
# Squared Frobenius norm:
np.conjugate(monopole_density_matrices[0] - density_matrices[200]).T @ (monopole_density_matrices[0] - density_matrices[200])

In [ ]:
def check_trace_distance(trace_distance_list):
    for trace_distance in trace_distance_list:
        if trace_distance > 1e-10:
            return False
    return True

In [ ]:
check_trace_distance(trace_distance_list)

In [ ]:
# Check the trace distance for element 200:
trace_distance_list[200] 

In [ ]:
fidelity(density_matrices[200], monopole_density_matrices[0])

In [ ]:
from qibo.backends import NumpyBackend
import numpy as np


def purity(state):
    backend = NumpyBackend()
    if len(state.shape) == 1:
        pur = backend.np.real(backend.calculate_vector_norm(state)) ** 2
    else:
        pur = backend.np.real(backend.np.trace(backend.np.matmul(state, state))) # tr(rho^2)
    return float(pur)

def fidelity1(state, target):
    backend = NumpyBackend()
    purity_state = purity(state) # purity of the density matrix 1 calculated using the backend. 
    purity_target = purity(target) # purity of the density matrix 2 calculated using the backend.

    # General formula for fidelity:

    # The eigenvalues and eigenvectors are used to reconstruct the square root of the density matrix 'state'.
    eigenvalues, eigenvectors = backend.calculate_eigenvectors(state)
    # Initialization of an empty state matrix to store the square root of it:
    state = np.zeros(state.shape, dtype=complex)
    #state = backend.cast(state, dtype=state.dtype)
    # Loop over the eigenvalues and eigenvectors to calculate the square root of the state matrix:
    # For each pair of eigenvalue and eigenvector, we calculate the outer product of the eigenvector
    # with itself.
    for eig, eigvec in zip(eigenvalues, backend.np.transpose(eigenvectors, (1, 0))):
        # Calculation of the square root of the state matrix using the spectral decomposition 
        # (since it is hermitian).
        matrix = backend.np.sqrt(eig) * backend.np.outer(eigvec, backend.np.conj(eigvec))
        #matrix = backend.cast(matrix, dtype=matrix.dtype)
        state = state + matrix # Summation of the spectral decomposition to obtain the square root of the state matrix.
        del matrix
    # Intermediate fidelity calculation: 
    fid = state @ target @ state # fid = state @ target @ state = sqrt(state) @ target @ sqrt(state)

    # We can REPEAT THE PROCESS again, now for fid, since fid is also hermitian:
    eigenvalues, eigenvectors = backend.calculate_eigenvectors(fid)
    # Initialization of an empty state matrix to store the square root of it:
    fid = np.zeros(state.shape, dtype=complex)
    fid = backend.cast(fid, dtype=fid.dtype)
    for eig, eigvec in zip(eigenvalues, backend.np.transpose(eigenvectors, (1, 0))):
        if backend.np.real(eig) > 1e-9: # Check if the eigenvalue is greater than 1e-8. WHY WE NEED THIS?
            matrix = backend.np.sqrt(eig) * backend.np.outer(eigvec, backend.np.conj(eigvec))
            #matrix = backend.cast(matrix, dtype=matrix.dtype)
            fid = fid + matrix
            del matrix
    # Final fidelity calculation:
    fid = backend.np.real(backend.np.trace(fid)) # Trace the resulting matrix

    return fid # tr(sqrt(fid)) [Nielsen and Chuang], but not the formula in the doc.

In [ ]:
from qibo.quantum_info import random_density_matrix
fidelity1(density_matrices[33], monopole_density_matrices[0])

In [ ]:
fidelity(density_matrices[33], monopole_density_matrices[0])

Fidelity check

In [ ]:
import numpy as np
from qibo.backends import NumpyBackend

def purity(state, backend=None):
    if len(state.shape) == 1:
        pur = backend.np.real(backend.calculate_vector_norm(state)) ** 2
    else:
        pur = backend.np.real(backend.np.trace(backend.np.matmul(state, state))) 
    return float(pur)

In [ ]:
def fidelity(state, target):

    # Check purity to check if they are mixed or pure states.
    if len(state.shape) == 2 and len(target.shape) == 2:
        purity_state = purity(state) # purity of the density matrix 1 calculated using the backend. 
        purity_target = purity(target) # purity of the density matrix 2 calculated using the backend.

        # IF BOTH STATES ARE MIXED: (Mixed state formula)
        # PRECISION_TOL = 1e-8 Tolerance for the probability sum check in the unitary channel. (See qibo>config.py)
        if (abs(purity_state - 1) > 1e-8 and abs(purity_target - 1) > 1e-8):
            # The eigenvalues and eigenvectors are used to reconstruct the square root of the density matrix 'state'.
            eigenvalues, eigenvectors = backend.calculate_eigenvectors(state)
            # Initialization of an empty state matrix to store the square root of it:
            state = np.zeros(state.shape, dtype=complex)
            state = backend.cast(state, dtype=state.dtype)
            # Loop over the eigenvalues and eigenvectors to calculate the square root of the state matrix:
            # For each pair of eigenvalue and eigenvector, we calculate the outer product of the eigenvector
            # with itself.
            for eig, eigvec in zip(eigenvalues, backend.np.transpose(eigenvectors, (1, 0))):
                # Calculation of the square root of the state matrix using the spectral decomposition 
                # (since it is hermitian).
                matrix = backend.np.sqrt(eig) * backend.np.outer(eigvec, backend.np.conj(eigvec))
                matrix = backend.cast(matrix, dtype=matrix.dtype)
                state = state + matrix # Summation of the spectral decomposition to obtain the square root of the state matrix.
                del matrix
            # Intermediate fidelity calculation: 
            fid = state @ target @ state # fid = state @ target @ state = sqrt(state) @ target @ sqrt(state)

            # We can REPEAT THE PROCESS again, now for fid, since fid is also hermitian:
            eigenvalues, eigenvectors = backend.calculate_eigenvectors(fid, hermitian=hermitian)
            # Initialization of an empty state matrix to store the square root of it:
            fid = np.zeros(state.shape, dtype=complex)
            fid = backend.cast(fid, dtype=fid.dtype)
            for eig, eigvec in zip(eigenvalues, backend.np.transpose(eigenvectors, (1, 0))):
                if backend.np.real(eig) > 1e-8: # Check if the eigenvalue is greater than 1e-8. WHY WE NEED THIS?
                    matrix = backend.np.sqrt(eig) * backend.np.outer(eigvec, backend.np.conj(eigvec))
                    matrix = backend.cast(matrix, dtype=matrix.dtype)
                    fid = fid + matrix
                    del matrix
            # Final fidelity calculation:
            fid = backend.np.real(backend.np.trace(fid)) # Trace the resulting matrix

            return fid # tr(sqrt(fid)) [Nielsen and Chuang], but not the formula in the doc.

    # IF ANY OF THE STATES IS PURE: 
    fid = (
        backend.np.abs(backend.np.matmul(backend.np.conj(state), target)) ** 2
        if len(state.shape) == 1 # State is a vector
        else backend.np.real(backend.np.trace(backend.np.matmul(state, target))) # matmul is used to perform matrix multiplication. It multiplies two arrays and returns the matrix product.
    )

    return fid